In [1]:
from flask import Flask
import pandas as pd
from flask import (request,
                   redirect,
                   url_for,
                   session,
                   render_template)

categories = ["sql", "programming", "web development", "art"]
from popularity import popularity_courses, similar_popular
import os

app = Flask(__name__)

In [16]:
import numpy as np

In [2]:
df = pd.read_csv("~/galvanize_notes/project/processed_data.csv")

In [5]:
df.columns

Index([u'Unnamed: 0', u'_class_x', u'content', u'course_id', u'created', u'id',
       u'modified', u'rating', u'title_x', u'user', u'userid', u'_class_y',
       u'image_125_H', u'image_240x135', u'image_480x270', u'input_features',
       u'is_paid', u'is_practice_test_course', u'lecture_search_result',
       u'predictive_score', u'price', u'published_title', u'relevancy_score',
       u'title_y', u'url', u'visible_instructors', u'user_rating_avg',
       u'user_review_count', u'course_review_count', u'course_rating_avg',
       u'instructor_name', u'instructor_title', u'new_course_id',
       u'new_user_id', u'course_category', u'Unnamed: 35'],
      dtype='object')

In [13]:
def popularity_courses(df):
    cold_users = df[df["user_review_count"] <=2]
    grouped_rating = pd.DataFrame(cold_users.groupby('published_title')['rating'].mean())
    grouped_rating['num of ratings'] = pd.DataFrame(cold_users.groupby('published_title')['rating'].count())
    grouped_rating.sort_values('num of ratings',ascending=False, inplace=True)
    grouped_rating.reset_index(inplace=True)
    top_courses = grouped_rating["published_title"].head(20)
    top_courses = pd.DataFrame(top_courses)
    top_courses = pd.merge(top_courses, df[["published_title", "price"]], on = "published_title")
    top_courses.drop_duplicates(inplace=True)
    return top_courses

In [14]:
top_courses = popularity_courses(df)

In [15]:
top_courses

,published_title,price
0,the-web-developer-bootcamp,$200
9092,aws-certified-solutions-architect-associate,$180
17139,unitycourse,$195
24674,the-complete-web-developer-course-2,$200
31684,the-complete-guide-to-angular-2,$190
37410,reikicourse,$195
42549,understand-javascript,$175
48443,selenium-with-c,$200
52790,java-the-complete-java-developer-course,$195
57577,complete-python-bootcamp,$195


In [17]:
course = np.random.choice(top_courses["published_title"].unique(), 1)

In [27]:
def similar_popular(topic, df):
    if topic not in popularity_courses(df)["published_title"].unique():
        return "please enter a valid course name from the table above"
    cold_users = df[df["user_review_count"] <=2]
    grouped_rating = pd.DataFrame(cold_users.groupby('published_title')['rating'].mean())
    grouped_rating['num of ratings'] = pd.DataFrame(cold_users.groupby('published_title')['rating'].count())
    course_mat = cold_users.pivot_table(index='userid',columns='published_title',values='rating')
    topic_user_ratings = course_mat[topic]
    similar_to_topic = course_mat.corrwith(topic_user_ratings)
    corr_topic = pd.DataFrame(similar_to_topic,columns=['correlation'])
    corr_topic.dropna(inplace=True)
    corr_topic.sort_values('correlation',ascending=False, inplace=True)
    
    corr_topic= corr_topic.join(grouped_rating['num of ratings'])
    corr_topic.reset_index(inplace=True)
    similar_topic = corr_topic[corr_topic['num of ratings']>100].sort_values('correlation',ascending=False).head()
    similar_topic = pd.DataFrame(similar_topic)
    similar_topic = pd.merge(similar_topic, df[["published_title", "price"]], on = "published_title")
    similar_topic.drop_duplicates(inplace=True)
    return similar_topic[["published_title", "price"]]
    


In [28]:
similar_popular(course, df)

,published_title,price
0,selenium-webdriver-page-objects,$75


In [30]:
len(df["course_category"].unique())

15

In [33]:
category = np.random.choice(df["course_category"].unique(), 1)

In [35]:
category[0]

'math'

In [40]:
catIwant = df[df["course_category"] == category[0]]
grouped_cat = pd.DataFrame(catIwant.groupby('published_title')['rating'].mean())
grouped_cat['num of ratings'] = pd.DataFrame(catIwant.groupby('published_title')['rating'].count())
grouped_cat.sort_values('num of ratings',ascending=False, inplace=True)
grouped_cat.reset_index(inplace=True)
print grouped_cat
grouped_cat = grouped_cat["published_title"].head(10)
top_courses = pd.DataFrame(grouped_cat)
top_courses = pd.merge(top_courses, df[["published_title", "price"]], on = "published_title")
top_courses.drop_duplicates(inplace=True)


        published_title    rating  num of ratings
0  integralcalc-algebra  4.653846             143
1             calculus1  4.694175             103


In [49]:
category[0]

'math'

In [69]:
def category_courses(category, df):
    catIwant = df[df["course_category"] == category[0]]
    grouped_cat = pd.DataFrame(catIwant.groupby('published_title')['rating'].mean())
    grouped_cat['num of ratings'] = pd.DataFrame(catIwant.groupby('published_title')['rating'].count())
    grouped_cat.sort_values('num of ratings',ascending=False, inplace=True)
    grouped_cat.reset_index(inplace=True)
    #grouped_cat = grouped_cat["published_title"].head(10)
    #cat_courses = pd.DataFrame(grouped_cat)
    cat_courses = pd.merge(grouped_cat, df[["published_title", "price"]], on = "published_title")
    cat_courses.drop_duplicates(inplace=True)
    return cat_courses[["published_title", "price"]].head(10)

In [70]:
category_courses(category, df)

,published_title,price
0,integralcalc-algebra,$100
143,calculus1,$150


In [50]:
catIwant = df[df["course_category"] == category[0]]

In [53]:
grouped_cat = pd.DataFrame(catIwant.groupby('published_title')['rating'].mean())

In [54]:
grouped_cat['num of ratings'] = pd.DataFrame(catIwant.groupby('published_title')['rating'].count())

In [56]:
grouped_cat.sort_values('num of ratings',ascending=False, inplace=True)

In [58]:
grouped_cat.reset_index(inplace=True)

In [62]:
cat_courses = pd.DataFrame(grouped_cat)

In [64]:
grouped_cat

,published_title,rating,num of ratings
0,integralcalc-algebra,4.653846,143
1,calculus1,4.694175,103


In [73]:
cat_course = category_courses(category, df)
cat_course.columns = ["Top courses", "price"]

In [74]:
cat_course

,Top courses,price
0,integralcalc-algebra,$100
143,calculus1,$150


In [80]:
def category_courses(category, df):
    catIwant = df[df["course_category"] == category]
    grouped_cat = pd.DataFrame(catIwant.groupby('published_title')['rating'].mean())
    grouped_cat['num of ratings'] = pd.DataFrame(catIwant.groupby('published_title')['rating'].count())
    grouped_cat.sort_values('num of ratings',ascending=False, inplace=True)
    grouped_cat.reset_index(inplace=True)
    grouped_cat = grouped_cat["published_title"].head(10)
    cat_courses = pd.DataFrame(grouped_cat)
    cat_courses = pd.merge(cat_courses, df[["published_title", "price"]], on = "published_title")
    cat_courses.drop_duplicates(inplace=True)
    return cat_courses[["published_title", "price"]].head(10)

In [85]:
type(category_courses("sql", df))

pandas.core.frame.DataFrame

In [78]:
df["course_category"].unique()

array(['web development', 'programming', 'sql', 'other', 'art',
       'data science', 'information technology',
       'software development / mobile application', 'business', 'adobe',
       'excel', 'math', 'writing', 'accounting', 'language'], dtype=object)